In [ ]:
import os
import pandas as pd
import numpy as np
from DataClasses import BloodData
from AssociativeAnalysis import run_cox_time_varying_analysis,get_blood_df_details,normalize_blood_df
from Utils import create_paths
import ConfigDataset as cfg 

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [ ]:
params=cfg.dataset_params
params.update({
    'is_predictive': False,
    'pred_future_timepoint': False,
    'blood_data_type': 'ROUTINE',
    'data_folder_path': r'./Data',
    'output_exp_path': r'./Results/routine_markers/cox_time_varying_analysis'
})
blood_data= BloodData(**params)
create_paths([blood_data.output_exp_path])

In [ ]:
blood_data.load_data()
blood_data.clean_data()
blood_data.select_included_exams()
blood_data.drop_unrequired_columns()
blood_data.selected_df.shape

In [ ]:
blood_data.calc_patients_stats()

In [ ]:
blood_df, blood_markers, df_markers, df_rest = get_blood_df_details(blood_data)
markers_scaled, df_markers_scaled = normalize_blood_df(df_markers, blood_markers)
blood_df = df_rest.join(df_markers_scaled)

PROGRESSION FREE SURVIVAL

In [ ]:
pfs_df = blood_df.drop(columns=['os', 'stop'])
# Drop the exams after progression
pfs_df = pfs_df.drop_duplicates(subset=["patient_ids", "stop_pfs"])
pfs_df = pfs_df.loc[np.where(pfs_df['stop_pfs'] > pfs_df['start'], True, False)]
pfs_df.columns

In [ ]:
cox_pfs_df=run_cox_time_varying_analysis(pfs_df,blood_data.output_exp_path,event_col='pfs', stop_col='stop_pfs')
cox_pfs_df.to_excel(os.path.join(blood_data.output_exp_path, 'pfs_cox_output.xlsx'))

OVERALL SURVIVAL

In [ ]:
os_df = blood_df.drop(columns=['pfs', 'stop_pfs'])
os_df.columns

In [ ]:
cox_os_df=run_cox_time_varying_analysis(os_df,blood_data.output_exp_path, event_col='os', stop_col='stop')
cox_os_df.to_excel(os.path.join(blood_data.output_exp_path, 'os_cox_output.xlsx'))